import the stock list excel file using pandas

In [107]:
import pandas as pd

stocks_list = pd.read_excel("stockslist.xlsx")
max_row = len(stocks_list.index)
# print(str(stocks_list["Holder"][0]))
stocks_list

,Stocks,Holder,Qty,PurchaseDate,PurchasePrice,Expectation
0,RCOM.BO,Ridhi,100,2021-05-20,1.75,300


Initiate our data frame with column names

In [119]:

my_columns = ["holder","qty","stock","purchase_price (₹) ","current_price (₹)","per_gain_loss (%)","per_growth (%)","annual_return (%)","tot_gain_loss (₹)","volume (lk)","mark_cap (cr)","week52high (₹)","week52low (₹)","purchase_date","expectance"]

final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,holder,qty,stock,purchase_price (₹),current_price (₹),per_gain_loss (%),per_growth (%),annual_return (%),tot_gain_loss (₹),volume (lk),mark_cap (cr),week52high (₹),week52low (₹),purchase_date,expectance


Lets get the details about the stock from the API calls

In [69]:
import yfinance as yf

stock = yf.Ticker("RCOM.BO")


In [ ]:
x = datetime.datetime.now()

In [102]:
def annual_return(pd,pp,cp):
    cd = datetime.datetime.now()
    gain_loss = (cp-pp)/pp*100
    days = (cd-pd).days
    
    if days/365>1:
        return round(gain_loss/(days/365),2)
    else:
        return round(gain_loss,2)
    
def gain_loss(pp,cp):
    return format((cp-pp)/pp*100,'.2f')

def growth(pp,cp):
    return format(cp/pp*100,'.2f')

In [70]:
print("cap - "+ str(stock.info["marketCap"])+ 
      "| vol - " + str(stock.info["volume"]) + 
      "| w52h - " + str(stock.info["fiftyTwoWeekHigh"])+ 
      "| w52l - " + str(stock.info["fiftyTwoWeekLow"]) + 
      "| regp - " + str(stock.info["regularMarketPrice"]) +
      "| prevc - "+ str(stock.info["previousClose"])
     )
# cap - 8891370496| vol - 41924106| w52h - 4.33| w52l - 0.62| regp - 3.24| prevc - 3.09

cap - 8891370496| vol - 42989107| w52h - 4.33| w52l - 0.62| regp - 3.24| prevc - 3.09


In [108]:
import yfinance as yf

for row in range(0,max_row):
    ticker = stocks_list["Stocks"][row]
    stock = yf.Ticker(ticker)
#     print(stock.info)
    final_dataframe = final_dataframe.append(
        pd.Series([
            stocks_list["Holder"][row],
            stocks_list["Qty"][row],
            ticker,
            stocks_list["PurchasePrice"][row],
            stock.info["regularMarketPrice"],
            gain_loss(pp = stocks_list["PurchasePrice"][row], cp = stock.info["regularMarketPrice"]),
            growth(pp = stocks_list["PurchasePrice"][row], cp = stock.info["regularMarketPrice"]),
            annual_return(pd = stocks_list["PurchaseDate"][row], pp = stocks_list["PurchasePrice"][row], cp = stock.info["regularMarketPrice"]),
            stock.info["regularMarketPrice"]-stocks_list["PurchasePrice"][row],
            round(stock.info["volume"]/100000,2),
            round(stock.info["marketCap"]/10000000,2),
            stock.info["fiftyTwoWeekHigh"],
            stock.info["fiftyTwoWeekLow"],
            stocks_list["PurchaseDate"][row],
            stocks_list["Expectation"][row],
        ], index=my_columns),
    ignore_index=True)

In [120]:
final_dataframe

,holder,qty,stock,purchase_price (₹),current_price (₹),per_gain_loss (%),per_growth (%),annual_return (%),tot_gain_loss (₹),volume (lk),mark_cap (cr),week52high (₹),week52low (₹),purchase_date,expectance


In [61]:
print(type(stocks_list["PurchaseDate"][0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [65]:
import datetime
pd = stocks_list["PurchaseDate"][0]
cd = datetime.datetime.now()

(cd-pd).days

19

In [67]:
round(stock.info["volume"]/100000,2)

424.96